In [174]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import urllib
import math
import copy
import pandas as pd
import numpy as np

In [175]:
# Determine the desired season start, end, number of games, and from or to the end date
start_year = "2017"
end_year = "2018"
from_to = "From"
games = 82

if(from_to == "From"):
    from_to_html = 1
else:
    from_to_html =-1

In [176]:
# Create a parser to scrape table rows only (no headers) from Puck On Net

class HTMLTableParser:

    def parse_url(self, url):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')
        return [(table['id'],self.parse_html_table(table))\
                for table in soup.find_all('table')]  

    def parse_html_table(self, table):
        n_columns = 0
        n_rows=0
        column_names = []

        # Find number of rows and columns
        # we also find the column titles if we can
        for row in table.find_all('tr'):

            # Determine the number of rows in the table
            td_tags = row.find_all('td')
            if len(td_tags) > 0:
                n_rows+=1
                if n_columns == 0:
                    # Set the number of columns for our table
                    n_columns = len(td_tags)


        columns = column_names if len(column_names) > 0 else range(0,n_columns)
        df = pd.DataFrame(columns = columns,
                          index= range(0,n_rows))
        row_marker = 0
        for row in table.find_all('tr'):
            column_marker = 0
            columns = row.find_all('td')
            for column in columns:
                df.iat[row_marker,column_marker] = column.get_text()
                column_marker += 1
            if len(columns) > 0:
                row_marker += 1

        return df

In [177]:
# GOALS 1
# Create an empty dataframe with the column headers for missed shots

goals_df = pd.DataFrame(columns=['Team', 'Game', 'Points','G Attempts ScoreAdj%', 
                                'G% Down 3+', 'G For Down 3+', 'G Agnst Down 3+', 
                                'G% Down 2', 'G For Down 2', 'G Agnst Down 2', 
                                'G% Down 1', 'G For Down 1', 'G Agnst Down 1', 
                                'G% Tied', 'G For Tied', 'G Agnst Tied', 
                                'G% Up 1', 'G For Up 1', 'G Agnst Up 1', 
                                'G% Up 2', 'G For Up 2', 'G Agnst Up 2', 
                                'G% Up 3+', 'G For Up 3+', 'G Agnst Up 3+'
                                ])

In [178]:
# GOALS 2
# Loop through tables for the cumulative totals after each w regular-season games in the season requested

w=0

for w in range(games):

    goals_url = "http://www.puckon.net/goals.php?s="+start_year+"-09-01&e="+end_year+"-06-30&f=0&ld="+str(from_to_html)+"&l="+str(w+1)+"&p=0"
    
    hp = HTMLTableParser()
    table = hp.parse_url(goals_url)[0][1] # Grabbing the table from the tuple
    table.columns = goals_df.columns
    goals_df = pd.concat([table, goals_df], axis=0, ignore_index=True)

    w = w+1
    

goals_made = goals_df
goals_made.head(32)

,Team,Game,Points,G Attempts ScoreAdj%,G% Down 3+,G For Down 3+,G Agnst Down 3+,G% Down 2,G For Down 2,G Agnst Down 2,...,G Agnst Tied,G% Up 1,G For Up 1,G Agnst Up 1,G% Up 2,G For Up 2,G Agnst Up 2,G% Up 3+,G For Up 3+,G Agnst Up 3+
0,ANA,82,101,53.21,60.00,9,6,60.71,17,11,...,59,61.02,36,23,54.05,20,17,42.11,8,11
1,ARI,82,70,45.55,50.00,11,11,47.37,18,20,...,72,35.00,21,39,54.55,12,10,72.73,8,3
2,BOS,82,112,54.91,54.55,6,5,62.07,18,11,...,62,58.06,36,26,64.86,24,13,58.33,14,10
3,BUF,82,62,39.92,48.65,18,19,34.69,17,32,...,71,47.92,23,25,31.25,5,11,16.67,1,5
4,CAR,82,83,46.69,30.43,7,16,42.86,12,16,...,69,44.87,35,43,50.00,15,15,33.33,4,8
5,CBJ,82,97,53.41,61.11,11,7,50.00,17,17,...,59,54.05,40,34,55.17,16,13,52.94,9,8
6,CGY,82,84,47.14,50.00,11,11,32.35,11,23,...,62,41.94,26,36,57.14,16,12,46.15,6,7
7,CHI,82,76,47.67,46.51,20,23,28.13,9,23,...,59,45.31,29,35,55.56,15,12,67.74,21,10
8,COL,82,95,50.36,44.44,12,15,44.83,13,16,...,52,52.11,37,34,57.14,20,15,58.62,17,12
9,DAL,82,92,53.00,58.82,10,7,37.93,11,18,...,57,54.84,34,28,60.00,15,10,70.37,19,8


In [179]:
# SHOTS 1
# Create an empty dataframe with the column headers for missed shots

shots_df = pd.DataFrame(columns=['Team', 'Game', 'Points','S Attempts ScoreAdj%', 
                                'S% Down 3+', 'S For Down 3+', 'S Agnst Down 3+', 
                                'S% Down 2', 'S For Down 2', 'S Agnst Down 2', 
                                'S% Down 1', 'S For Down 1', 'S Agnst Down 1', 
                                'S% Tied', 'S For Tied', 'S Agnst Tied', 
                                'S% Up 1', 'S For Up 1', 'S Agnst Up 1', 
                                'S% Up 2', 'S For Up 2', 'S Agnst Up 2', 
                                'S% Up 3+', 'S For Up 3+', 'S Agnst Up 3+'
                                ])

In [180]:
# SHOTS 2
# Loop through tables for the cumulative totals after each x regular-season games in the season requested

x=0

for x in range(games):

    shots_url = "http://www.puckon.net/shots.php?s="+start_year+"-09-01&e="+end_year+"-06-30&f=0&ld="+str(from_to_html)+"&l="+str(x+1)+"&p=0"
    
    hp = HTMLTableParser()
    table = hp.parse_url(shots_url)[0][1] # Grabbing the table from the tuple
    table.columns = shots_df.columns
    shots_df = pd.concat([table, shots_df], axis=0, ignore_index=True)

    x = x+1
    

shots_made = shots_df
shots_made.head(32)

,Team,Game,Points,S Attempts ScoreAdj%,S% Down 3+,S For Down 3+,S Agnst Down 3+,S% Down 2,S For Down 2,S Agnst Down 2,...,S Agnst Tied,S% Up 1,S For Up 1,S Agnst Up 1,S% Up 2,S For Up 2,S Agnst Up 2,S% Up 3+,S For Up 3+,S Agnst Up 3+
0,ANA,82,101,48.46,61.82,68,42,58.11,154,111,...,718,43.50,338,439,41.14,195,279,42.32,102,139
1,ARI,82,70,47.38,54.22,135,114,52.67,197,177,...,802,41.11,296,424,42.27,93,127,38.98,46,72
2,BOS,82,112,53.50,68.79,108,49,59.41,202,138,...,712,48.44,326,347,51.13,136,130,50.17,147,146
3,BUF,82,62,47.63,56.27,211,164,55.01,214,175,...,729,42.93,255,339,38.51,67,107,26.25,21,59
4,CAR,82,83,53.91,53.33,144,126,62.54,187,112,...,743,48.87,367,384,49.82,139,140,47.62,90,99
5,CBJ,82,97,52.19,53.90,76,65,62.09,267,163,...,761,44.86,380,467,45.45,135,162,45.12,111,135
6,CGY,82,84,52.59,62.00,155,95,58.77,201,141,...,756,48.37,327,349,46.64,139,159,52.23,82,75
7,CHI,82,76,51.20,59.47,267,182,62.09,208,127,...,805,47.52,326,360,44.53,122,152,50.55,139,136
8,COL,82,95,47.19,56.25,90,70,56.41,132,102,...,745,44.71,355,439,35.44,129,235,40.18,88,131
9,DAL,82,92,51.32,60.50,144,94,58.10,165,119,...,674,44.11,292,370,37.63,70,116,47.76,128,140


In [181]:
# MISSED SHOTS 1
# Create an empty dataframe with the column headers for missed shots

missed_df = pd.DataFrame(columns=['Team', 'Game', 'Points','M Attempts ScoreAdj%', 
                                'M% Down 3+', 'M For Down 3+', 'M Agnst Down 3+', 
                                'M% Down 2', 'M For Down 2', 'M Agnst Down 2', 
                                'M% Down 1', 'M For Down 1', 'M Agnst Down 1', 
                                'M% Tied', 'M For Tied', 'M Agnst Tied', 
                                'M% Up 1', 'M For Up 1', 'M Agnst Up 1', 
                                'M% Up 2', 'M For Up 2', 'M Agnst Up 2', 
                                'M% Up 3+', 'M For Up 3+', 'M Agnst Up 3+'
                                ])

In [182]:
# MISSED SHOTS 2
# Loop through tables for the cumulative totals after each x regular-season games in the season requested

y=0

for y in range(games):

    missed_url = "http://www.puckon.net/misses.php?s="+start_year+"-09-01&e="+end_year+"-06-30&f=0&ld="+str(from_to_html)+"&l="+str(y+1)+"&p=0"

    hp = HTMLTableParser()
    table = hp.parse_url(missed_url)[0][1] # Grabbing the table from the tuple
    table.columns = missed_df.columns
    missed_df = pd.concat([table, missed_df], axis=0, ignore_index=True)

    y = y+1
    

missed_shots = missed_df
missed_shots.head(32)

,Team,Game,Points,M Attempts ScoreAdj%,M% Down 3+,M For Down 3+,M Agnst Down 3+,M% Down 2,M For Down 2,M Agnst Down 2,...,M Agnst Tied,M% Up 1,M For Up 1,M Agnst Up 1,M% Up 2,M For Up 2,M Agnst Up 2,M% Up 3+,M For Up 3+,M Agnst Up 3+
0,ANA,82,101,49.47,51.67,31,29,55.04,71,58,...,304,45.50,182,218,44.44,104,130,45.69,53,63
1,ARI,82,70,45.07,51.75,59,55,47.40,82,91,...,437,38.44,133,213,40.40,40,59,31.82,21,45
2,BOS,82,112,53.75,62.26,33,20,63.71,79,45,...,298,46.50,146,168,53.19,75,66,45.83,66,78
3,BUF,82,62,45.20,54.09,86,73,52.76,86,77,...,277,40.27,89,132,29.23,19,46,31.25,10,22
4,CAR,82,83,53.56,49.61,63,64,49.64,68,69,...,314,49.69,159,161,46.09,53,62,37.68,26,43
5,CBJ,82,97,53.41,66.22,49,25,57.65,98,72,...,283,48.14,155,167,45.60,57,68,44.34,47,59
6,CGY,82,84,54.45,67.39,93,45,60.47,104,68,...,349,48.73,153,161,49.60,62,63,52.86,37,33
7,CHI,82,76,50.59,63.48,113,65,56.38,84,65,...,313,49.38,120,123,39.78,37,56,37.74,40,66
8,COL,82,95,46.76,43.82,39,50,48.96,47,49,...,315,49.30,141,145,43.23,67,88,41.24,40,57
9,DAL,82,92,51.30,60.67,54,35,54.05,60,51,...,270,47.76,128,140,41.67,40,56,44.03,59,75


In [183]:
# Blocked SHOTS 1
# Create an empty dataframe with the column headers for missed shots

blocked_df = pd.DataFrame(columns=['Team', 'Game', 'Points','B Attempts Score Adj%', 
                                'B% Down 3+', 'B For Down 3+', 'B Agnst Down 3+', 
                                'B% Down 2', 'B For Down 2', 'B Agnst Down 2', 
                                'B% Down 1', 'B For Down 1', 'B Agnst Down 1', 
                                'B% Tied', 'B For Tied', 'B Agnst Tied', 
                                'B% Up 1', 'B For Up 1', 'B Agnst Up 1', 
                                'B% Up 2', 'B For Up 2', 'B Agnst Up 2', 
                                'B% Up 3+', 'B For Up 3+', 'B Agnst Up 3+'
                                ])

In [184]:
# BLOCKED SHOTS 2
# Loop through tables for the cumulative totals after each x regular-season games in the season requested

z=0

for z in range(games):
    blocked_url = "http://www.puckon.net/blocks.php?s="+start_year+"-09-01&e="+end_year+"-06-30&f=0&ld="+str(from_to_html)+\
    "&l="+str(z+1)+"&p=0"
    
    hp = HTMLTableParser()
    table = hp.parse_url(blocked_url)[0][1] # Grabbing the table from the tuple
    table.columns = blocked_df.columns
    blocked_df = pd.concat([table, blocked_df], axis=0, ignore_index=True)

    z = z+1
    

blocked_shots = blocked_df
blocked_shots.head(32)

,Team,Game,Points,B Attempts Score Adj%,B% Down 3+,B For Down 3+,B Agnst Down 3+,B% Down 2,B For Down 2,B Agnst Down 2,...,B Agnst Tied,B% Up 1,B For Up 1,B Agnst Up 1,B% Up 2,B For Up 2,B Agnst Up 2,B% Up 3+,B For Up 3+,B Agnst Up 3+
0,ANA,82,101,48.92,71.43,40,16,62.07,108,66,...,369,40.99,166,239,42.04,95,131,32.11,35,74
1,ARI,82,70,51.67,60.34,70,46,57.35,117,87,...,384,45.52,193,231,45.38,54,65,42.17,35,48
2,BOS,82,112,54.10,64.56,51,28,66.23,100,51,...,342,45.45,170,204,48.46,63,67,48.10,76,82
3,BUF,82,62,46.41,56.65,98,75,55.88,95,75,...,332,37.36,99,166,34.09,30,58,28.57,12,30
4,CAR,82,83,58.21,67.46,85,41,72.79,99,37,...,348,51.58,196,184,49.01,74,77,46.34,38,44
5,CBJ,82,97,48.85,62.90,39,23,58.59,116,82,...,361,37.29,154,259,33.54,53,105,45.80,60,71
6,CGY,82,84,53.62,68.25,86,40,64.90,135,73,...,395,47.45,158,175,50.40,63,62,38.75,31,49
7,CHI,82,76,56.21,69.84,176,76,66.05,142,73,...,380,52.14,195,179,44.70,59,73,46.07,82,96
8,COL,82,95,49.50,52.63,50,45,60.29,82,54,...,376,49.64,205,208,33.99,69,134,33.88,41,80
9,DAL,82,92,51.08,56.92,74,56,63.33,114,66,...,398,45.85,177,209,27.36,29,77,42.77,71,95


In [185]:
goals_shots_missed_blocked= pd.concat([goals_made, shots_made, missed_shots, blocked_shots], axis=1)
goals_shots_missed_blocked.head(32)

,Team,Game,Points,G Attempts ScoreAdj%,G% Down 3+,G For Down 3+,G Agnst Down 3+,G% Down 2,G For Down 2,G Agnst Down 2,...,B Agnst Tied,B% Up 1,B For Up 1,B Agnst Up 1,B% Up 2,B For Up 2,B Agnst Up 2,B% Up 3+,B For Up 3+,B Agnst Up 3+
0,ANA,82,101,53.21,60.00,9,6,60.71,17,11,...,369,40.99,166,239,42.04,95,131,32.11,35,74
1,ARI,82,70,45.55,50.00,11,11,47.37,18,20,...,384,45.52,193,231,45.38,54,65,42.17,35,48
2,BOS,82,112,54.91,54.55,6,5,62.07,18,11,...,342,45.45,170,204,48.46,63,67,48.10,76,82
3,BUF,82,62,39.92,48.65,18,19,34.69,17,32,...,332,37.36,99,166,34.09,30,58,28.57,12,30
4,CAR,82,83,46.69,30.43,7,16,42.86,12,16,...,348,51.58,196,184,49.01,74,77,46.34,38,44
5,CBJ,82,97,53.41,61.11,11,7,50.00,17,17,...,361,37.29,154,259,33.54,53,105,45.80,60,71
6,CGY,82,84,47.14,50.00,11,11,32.35,11,23,...,395,47.45,158,175,50.40,63,62,38.75,31,49
7,CHI,82,76,47.67,46.51,20,23,28.13,9,23,...,380,52.14,195,179,44.70,59,73,46.07,82,96
8,COL,82,95,50.36,44.44,12,15,44.83,13,16,...,376,49.64,205,208,33.99,69,134,33.88,41,80
9,DAL,82,92,53.00,58.82,10,7,37.93,11,18,...,398,45.85,177,209,27.36,29,77,42.77,71,95


In [186]:
# Add a column to specify the season's ending year, and a column to specify regular season or playoffs
goals_shots_missed_blocked['Season End'] = end_year
goals_shots_missed_blocked['Season Type'] = 'Regular'
goals_shots_missed_blocked['From / To'] = from_to
goals_shots_missed_blocked.head()

,Team,Game,Points,G Attempts ScoreAdj%,G% Down 3+,G For Down 3+,G Agnst Down 3+,G% Down 2,G For Down 2,G Agnst Down 2,...,B Agnst Up 1,B% Up 2,B For Up 2,B Agnst Up 2,B% Up 3+,B For Up 3+,B Agnst Up 3+,Season End,Season Type,From / To
0,ANA,82,101,53.21,60.00,9,6,60.71,17,11,...,239,42.04,95,131,32.11,35,74,2018,Regular,From
1,ARI,82,70,45.55,50.00,11,11,47.37,18,20,...,231,45.38,54,65,42.17,35,48,2018,Regular,From
2,BOS,82,112,54.91,54.55,6,5,62.07,18,11,...,204,48.46,63,67,48.10,76,82,2018,Regular,From
3,BUF,82,62,39.92,48.65,18,19,34.69,17,32,...,166,34.09,30,58,28.57,12,30,2018,Regular,From
4,CAR,82,83,46.69,30.43,7,16,42.86,12,16,...,184,49.01,74,77,46.34,38,44,2018,Regular,From


In [187]:
# Export file as a CSV, without the Pandas index, but with the header
csv_write_path = "Desktop/nhl_predictor/combined_shot_"+end_year+"_"+from_to+"-season_end.csv"
goals_shots_missed_blocked.to_csv(csv_write_path, index=False, header=True)